# Introduction

**pytorch**是python的一個程式庫，就像是處理天文數據會用astropy那樣，**pytorch**是專門拿來開發AI的工具，可想像成可以被GPU加速的numpy

有二大特點，使其能夠很好的作為深度學習的框架：
1. 利用 **張量(tensor)** 作為處理資料的基本單位，並且可被丟到GPU上面做數學運算
2. **自動微分引擎(Autograd)**：我們在寫程式只要定義好forward要怎麼算，系統就會自動幫我們紀錄路徑，再透過連鎖律算出所有參數的微分，從而修正模型參數

# 前置準備

函式庫更新可能追不上python版本更新，所以建議使用比最新版本低1-2版本的python(我是用3.10)；有CUDA相容的GPU的建議就安裝支援GPU的版本，嘗試看看用GPU跑程式  
PS: CUDA是個運算平台，負責*指揮*GPU去看懂程式

關於要在terminal打甚麼指令安裝pytorch，因為每個人系統不一樣，因此請見官網：
`https://pytorch.org/`  
然後這邊採用PyTorch 2.4.0，所以建議把指令的torch改成`torch==2.4.0`

In [1]:
import torch

# 張量簡介

- 0階張量: scalar
- 1階張量: vector
- 2階張量: Matrix
- etc.

In [2]:
# create a tensor
tensor_0d = torch.tensor(1)
tensor_1d = torch.tensor([1, 2, 3])
tensor_2d = torch.tensor([[1, 2, 3],
                          [4, 5, 6]])

for i in range(3):
    if i == 0:
        tensor = tensor_0d
    elif i == 1:
        tensor = tensor_1d
    else:
        tensor = tensor_2d
    print(f"{i}D Tensor:", tensor,tensor.shape,"\n",sep="\n")  

0D Tensor:
tensor(1)
torch.Size([])


1D Tensor:
tensor([1, 2, 3])
torch.Size([3])


2D Tensor:
tensor([[1, 2, 3],
        [4, 5, 6]])
torch.Size([2, 3])




In [3]:
# from the output, we can see that the 2D tensor has shape (2, 3),
# if we want to reshape it to (3, 2), we can do as follows:

# reshape tensors(take 2D tensor as example)
tensor = tensor_2d

# there are two ways to reshape a tensor
# 1. using reshape() method
re_tensor_1 = tensor.reshape(3, 2)

# 2. using view() method
re_tensor_2 = tensor.view(3, 2)

print(re_tensor_1,"\n\n",re_tensor_2)

tensor([[1, 2],
        [3, 4],
        [5, 6]]) 

 tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [4]:
# to transpose a tensor, using T attribute
tensor_1 = torch.tensor([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 9]])
tensor_transposed_1 = tensor_1.T

print(tensor_1,"\n\n",tensor_transposed_1)

print("=======================")

# try anothor tensor
tensor_2 = torch.tensor([[1, 2, 3],
                         [4, 5, 6]])
tensor_transposed_2 = tensor_2.T

print(tensor_2, "\n\n", tensor_transposed_2)



tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]]) 

 tensor([[1, 4, 7],
        [2, 5, 8],
        [3, 6, 9]])
tensor([[1, 2, 3],
        [4, 5, 6]]) 

 tensor([[1, 4],
        [2, 5],
        [3, 6]])


In [7]:
# if we want to muptliply two tensors, there are two methods
tensor_a = torch.tensor([[1, 2, 3],
                         [4, 5, 6]])
tensor_b = torch.tensor([[1, 2],
                         [3, 4],
                         [5, 6]])

# 1. using .matmul()
tensor_c = tensor_a.matmul(tensor_b)
print(tensor_c)

print("=======================")

# 2. using @
tensor_c = tensor_a@tensor_b
print(tensor_c)

tensor([[22, 28],
        [49, 64]])
tensor([[22, 28],
        [49, 64]])


## 大家都要學會的招數

```.to()``` 可以讓張量運算指定到GPU上運行

In [ ]:
x = torch.tensor([1, 2, 3])
x = x.to("cuda")            # Move tensor to GPU

### 大家跑比賽

In [19]:
import time
# 謝謝G學長教我怎麼算時間
N = 8000

tensor_a = torch.randn(N, N)
tensor_b = torch.randn(N, N)
# move to GPU
tensor_a_GPU = tensor_a.to("cuda")
tensor_b_GPU = tensor_b.to("cuda")

# 預熱 (Warm-up)
_ = tensor_a @ tensor_b 
_ = tensor_a_GPU @ tensor_b_GPU
torch.cuda.synchronize()        # 等待預熱完成

# ============================
# CPU 系列
# ============================
start = time.time()

result_CPU = tensor_a@tensor_b

end = time.time()

cpu_time_ms = (end - start) * 1000  # s -> ms

print(f"CPU time: {cpu_time_ms:.2f} ms")

# ============================
# GPU 系列
# ============================
# create events
start_GPU = torch.cuda.Event(enable_timing=True)
end_GPU = torch.cuda.Event(enable_timing=True)

start_GPU.record()

result_GPU = tensor_a_GPU@tensor_b_GPU

end_GPU.record()

# 等待GPU跑完
torch.cuda.synchronize()

gpu_time_ms = start_GPU.elapsed_time(end_GPU)

print(f"GPU time:{gpu_time_ms:.2f} ms")

CPU time: 3791.07 ms
GPU time:795.06 ms
